In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Connection au Workspace

In [ ]:
# load workspace configuration from the config.json file in the current folder.
subscription_id = "votre clé"
resource_group = "ProjectGroup3"
workspace_name = "Christophe_ML"
ws = Workspace(subscription_id, resource_group, workspace_name)
print(ws.name, ws.location, ws.resource_group, sep='\t')

### Création d'un blob storage

In [ ]:
from azure.storage.blob import BlockBlobService

# Connect to our blob via the BlobService
blob_service = BlockBlobService('christophestorage', 'clé du storage')

# Create a container
blob_service.create_container('seloger')

### Upload du csv dans le blob

In [ ]:
from azure.storage.blob import Blob

blob_service.create_blob_from_path('seloger', 'biens_features', 'biens_features.csv', content_settings=None, metadata=None, validate_content=False, progress_callback=None, max_connections=2, lease_id=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None)

### Liaison du blob storage au workspace de AMLS

In [ ]:
from azureml.data.data_reference import DataReference
from msrest.exceptions import HttpOperationError

blob_datastore_name='christophestore'
account_name=os.getenv("BLOB_ACCOUNTNAME_62", "christophestorage") # Storage account name
container_name=os.getenv("BLOB_CONTAINER_62", "seloger") # Name of Azure blob container
account_key=os.getenv("BLOB_ACCOUNT_KEY_62", "clé du storage") # Storage account key

try:
    blob_datastore = Datastore.get(ws, blob_datastore_name)
    print("Found Blob Datastore with name: %s" % blob_datastore_name)
except HttpOperationError:
    blob_datastore = Datastore.register_azure_blob_container(
        workspace=ws,
        datastore_name=blob_datastore_name,
        account_name=account_name, # Storage account name
        container_name=container_name, # Name of Azure blob container
        account_key=account_key) # Storage account key
    print("Registered blob datastore with name: %s" % blob_datastore_name)

blob_data_ref = DataReference(
    datastore=blob_datastore,
    data_reference_name="blob_test_data",
    path_on_datastore="testdata")

### Création du dataset à partir du blob

In [ ]:
from azureml.core import Dataset

datastore = Datastore.get(ws, 'christophestore')
datastore_paths = [(datastore, 'biens_features')]
biens_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

### Enregistrement du dataset

In [ ]:
biens_ds.register(workspace =ws, name ="biens_dataset", description="test", create_new_version=True)

### Récupération du dataset

In [ ]:
import azureml.core
from azureml.core import Workspace, Dataset

# Get a dataset from the workspace datasets collection
ds1 = ws.datasets['biens_dataset']
ds1

